<center><h1>Mini Project 2</h1></center>
<center><h2>Watchanan Chantapakul (<a href="mailto:wcgzm@umsystem.edu"><code>wcgzm</code></a>)</h2></center>
<hr />

In [1]:
%matplotlib widget
import numpy as np
import sympy
import scipy
from scipy import spatial
np.random.seed(7720)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

A two-class dataset has gaussian likelihood functions and priors $P(\omega_1) = 4P(\omega_2)$. Let the parameters of the likelihoods be $\mu_1 = \begin{pmatrix} 7 \\ 1 \end{pmatrix}$, $\mu_2 = \begin{pmatrix} 1 \\ 7 \end{pmatrix}$ and $\Sigma_1 = \Sigma_2 = \begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}$

In [2]:
C = 2 # number of classes
d = 2 # dimension
priors = [4/5, 1/5]
"P(𝜔1) = 4P(𝜔2)", priors[0] == 4 * priors[1]

('P(𝜔1) = 4P(𝜔2)', True)

In [3]:
means = np.array([[7, 1], [1, 7]])
"mu1", means[0], "mu2", means[1]

('mu1', array([7, 1]), 'mu2', array([1, 7]))

In [4]:
cov = np.zeros((C, d, d))
for c in range(C):
    cov[c]= np.array([[3.1, 0], [0, 2.6]])
"Sigma1, Sigma2 =", cov

('Sigma1, Sigma2 =',
 array([[[3.1, 0. ],
         [0. , 2.6]],
 
        [[3.1, 0. ],
         [0. , 2.6]]]))

In [5]:
means[0]

array([7, 1])

In [6]:
# x = np.random.multivariate_normal(mean=means[c], cov=cov[c], size=C)
# print("x1", x[0])
# print("x2", x[1])

In [7]:
# fig = plt.figure(figsize=(12, 6))
# ax = fig.add_subplot(111)
# viz_X1 = np.random.multivariate_normal(mean=means[0], cov=cov, size=1000)
# viz_X2 = np.random.multivariate_normal(mean=means[1], cov=cov, size=1000)
# ax.scatter(viz_X1[:, 0], viz_X1[:, 1], label="$\omega_1$")
# ax.scatter(viz_X2[:, 0], viz_X2[:, 1], label="$\omega_2$")
# ax.scatter(means[0, 0], means[0, 1], marker="*", s=250, label="$\mu_1$")
# ax.scatter(means[1, 0], means[1, 1], marker="*", s=250, label="$\mu_2$")
# ax.scatter(x[0, 0], x[0, 1], marker="x", color="red", s=100, label="Test sample 1")
# ax.scatter(x[1, 0], x[1, 1], marker="x", color="magenta", s=100, label="Test sample 2")
# ax.set_aspect(1)
# ax.legend()
# plt.show()

# Question A
a) Write a generic Matlab function[<sup>1</sup>](#fn1) to compute the Mahalanobis distances between two arbitrary samples $\vec{x_1}$ and $\vec{x_2}$ or the distance between a sample $\vec{x_1}$ and the center of any given Gaussian distribution with covariance $\Sigma$, mean $\mu$, and dimension $d$.

<span id="fn1"><sup>1</sup> you may use any computer language/package, but you may NOT use any function other than the basic operations: i.e. +, -, *, / (for scalars, vectors, or matrices)</span>

## Solution

- $\mathbf{A}$ = a matrix
- $i$ = row
- $j$ = column

### $(i, j)$-Minor of a Matrix
$$ \mathbf{M}_{i, j} = \operatorname{det}((\mathbf{A}_{p, q})_{p \neq i, q \neq j}) = |(\mathbf{A}_{p, q})_{p \neq i, q \neq j}| $$

In [8]:
def submatrix(A, i, j):
    B = []
    for row in A[:i] + A[i+1:]:
        B.append(row[:j]+ row[j+1:])
    return B

def minor(A, i, j):
    return determinant(submatrix(A, i, j))

In [9]:
A = [[1,3,1,4],[3,9,5,15],[0,2,1,1],[0,4,2,3]]
i = 1
j = 3
np_submatrix = np.delete(np.delete(A,i,axis=0), j, axis=1)
my_submatrix = np.array(submatrix(A, i, j))
print("Numpy's delete == transpose(A)", np.allclose(np_submatrix, my_submatrix))
print("------------------------------------")
print("np =\n", np_submatrix)
print("submatrix =\n", my_submatrix)

Numpy's delete == transpose(A) True
------------------------------------
np =
 [[1 3 1]
 [0 2 1]
 [0 4 2]]
submatrix =
 [[1 3 1]
 [0 2 1]
 [0 4 2]]


### Determinant
$$ \operatorname{det}(A) = |\mathbf{A}| = \sum_{i=0, j} (-1)^{i+j} \cdot \mathbf{A}_{i, j} \cdot \mathbf{M}_{i, j} = \sum_{i=0, j} (-1)^{j} \cdot \mathbf{A}_{0, j} \cdot \mathbf{M}_{0, j} $$

In [10]:
def determinant(A):
    if len(A) == 2:
        det = (A[0][0] * A[1][1]) - (A[1][0] * A[0][1])
        return det
    else:
        det = 0
        
        i = 0
        for j in range(len(A)):
            det += ((-1) ** (i + j)) * A[i][j] * minor(A, i, j)
        return det

In [11]:
A = [[1,3,1,4],[3,9,5,15],[0,2,1,1],[0,4,2,3]]

np_det = np.linalg.det(A)
my_det = determinant(A)
print("np.lin.det(A) == determinant(A)", np.allclose(np_det, my_det))
print("------------------------------------")
print("np =", np_det)
print("determinant =", my_det)

np.lin.det(A) == determinant(A) True
------------------------------------
np = -3.999999999999999
determinant = -4


### Matrix of Cofactors
$$ \mathbf{C}_{i, j} = (–1)^{i+j}  \mathbf{M}_{i, j} $$

In [12]:
def cofactor(m, i, j):
    return ((-1) ** (i + j)) * m

### Transpose
$$ \mathbf{A}^{\mathsf{T}}_{i, j} = \mathbf{A}_{j, i} $$

In [13]:
def transpose(m):
    return list(map(list,zip(*m)))

In [14]:
A = np.array([[1,3,1,4],[3,9,5,15],[0,2,1,1],[0,4,2,3]])
np_t = A.T
my_t = np.array(transpose(A))
print("np.transpose(A) == transpose(A)", np.allclose(np_t, my_t))
print("------------------------------------")
print("np =\n", np_t)
print("transpose =\n", my_t)

np.transpose(A) == transpose(A) True
------------------------------------
np =
 [[ 1  3  0  0]
 [ 3  9  2  4]
 [ 1  5  1  2]
 [ 4 15  1  3]]
transpose =
 [[ 1  3  0  0]
 [ 3  9  2  4]
 [ 1  5  1  2]
 [ 4 15  1  3]]


### Adjugate Matrix
$$ \operatorname{adj}(\mathbf{A}) = \mathbf{C}^{\mathsf{T}} $$

In [15]:
def adjugate(A):
    N = len(A)
    adj = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            adj[i, j] = cofactor(minor(A, i, j), i, j)
    return transpose(adj)

In [16]:
A = [[1,3,1,4],[3,9,5,15],[0,2,1,1],[0,4,2,3]]
sympy_adj = np.array(sympy.Matrix(A).adjugate(), dtype=float)
my_adj = np.array(adjugate(A), dtype=float)

print("Sympy's adjugate == adjugate(A)", np.allclose(sympy_adj, my_adj))
print("------------------------------------")
print("sympy =\n", sympy_adj)
print("adjugate =\n", my_adj)

Sympy's adjugate == adjugate(A) True
------------------------------------
sympy =
 [[ -1.  -1. -20.  13.]
 [ -3.   1.   0.  -1.]
 [  6.  -2. -12.   6.]
 [  0.   0.   8.  -4.]]
adjugate =
 [[ -1.  -1. -20.  13.]
 [ -3.   1.   0.  -1.]
 [  6.  -2. -12.   6.]
 [  0.   0.   8.  -4.]]


### Inverse Matrix
$$ \mathbf{A}^{-1} = |\mathbf{A}|^{-1} \cdot \operatorname{adj}(\mathbf{A}) = \frac{\operatorname{adj}(\mathbf{A})}{|\mathbf{A}|} $$

In [17]:
def inverse_matrix(A):
    if len(A) == 2:
        inv_A = [
            [A[1][1], -A[0][1]],
            [-A[1][0], A[0][0]]
        ]
        return np.array(inv_A) / determinant(A)
    else:
        adj = adjugate(A)
        return np.array(adj) / determinant(A)

In [18]:
# A = [[1, 2], [3, 4]]
A = [[1,3,1,4],[3,9,5,15],[0,2,1,1],[0,4,2,3]]
np_inv = np.linalg.inv(A)
my_inv = inverse_matrix(A)

print("np.linalg.inv(A) == inverse_matrix(A)", np.allclose(np_inv, my_inv))
print("------------------------------------------")
print("np.linalg.inv(A) =\n", np_inv)
print("inverse_matrix(A) =\n", my_inv)

np.linalg.inv(A) == inverse_matrix(A) True
------------------------------------------
np.linalg.inv(A) =
 [[ 2.5000000e-01  2.5000000e-01  5.0000000e+00 -3.2500000e+00]
 [ 7.5000000e-01 -2.5000000e-01 -4.4408921e-16  2.5000000e-01]
 [-1.5000000e+00  5.0000000e-01  3.0000000e+00 -1.5000000e+00]
 [-0.0000000e+00 -0.0000000e+00 -2.0000000e+00  1.0000000e+00]]
inverse_matrix(A) =
 [[ 0.25  0.25  5.   -3.25]
 [ 0.75 -0.25 -0.    0.25]
 [-1.5   0.5   3.   -1.5 ]
 [-0.   -0.   -2.    1.  ]]


### Mahalanobis Distance
$$ r^2 = (\vec{x} - \vec{y})^{\mathsf{T}} \mathbf{\Sigma}^{-1} (\vec{x} - \vec{y}) $$

In [19]:
def mahalanobis_distance(x, mean, cov):
    a = np.array(x) - np.array(mean)
    r2 = a.T @ inverse_matrix(cov) @ a
    return np.sqrt(r2)

In [20]:
# my_mahalanobis = np.zeros((len(x), C))
# scipy_mahalanobis = np.zeros_like(my_mahalanobis)
# for i in range(len(x)):
#     for c in range(C):
#         my_mahalanobis[i, c] = mahalanobis_distance(x[i], means[c], cov[c])
#         scipy_mahalanobis[i, c] = scipy.spatial.distance.mahalanobis(x[i], means[c], VI=np.linalg.inv(cov[c]))
# print(my_mahalanobis)
# print(scipy_mahalanobis)

## Question B

b) Write another Matlab function[<sup>1</sup>](#fn11) to call the function above and compute the discriminant function with the following generic form 
$$ g_i(x) = -\frac{1}{2}(x-\mu_i)^t\Sigma_i^{-1}(x-\mu_i)-\frac{d}{2}\ln (2\pi) - \frac{1}{2} \ln |\Sigma_i| + \ln P(\omega_i) $$
also for any given $d$ dimensional data, mean, covariance matrix and prior probabilities.

<span id="fn11"><sup>1</sup> you may use any computer language/package, but you may NOT use any function other than the basic operations: i.e. +, -, *, / (for scalars, vectors, or matrices)</span>

In [21]:
def discriminant_fx(x, mean, cov, prior):
    d = len(x)
    return (-0.5 * mahalanobis_distance(x, mean, cov)) - ((d/2) * np.log(2 * np.pi)) - (0.5 * np.log(determinant(cov))) + np.log(prior)

c) write a Matlab program that generates (say, 1000) samples from the two classes with the parameters in part a); and plot the two classes in 3D. (your plot should be similar to figure 2.10 (b) in the
textbook). The class samples above MUST be created from a Gaussian distribution with $N(~0, \mathbf{I})$ (ie. use the concept of whitening in an inverse manner).[<sup>2</sup>](#fn2)

<span id="fn2"><sup>2</sup>
    That is, do NOT use a Matlab Toolbox or any other library function, to generate the distributions above directly from the parameters in part a). You MUST do a “dewhitening” instead. In that case, the following Matlab functions can still be useful for this assignment: <code>randn(), peaks(), meshgrid(), surf(), and mesh()</code>
</span>

In [49]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
sample_colors = ("tab:blue", "tab:pink")
mean_colors = ("b", "m")

class1_samples = np.random.multivariate_normal(mean=[0, 0], cov=np.identity(2), size=1000)
print(class1_samples[0])
class1_samples = class1_samples
class1_samples += means[0]
print(class1_samples[0])

# x = np.arange(samples[:, 0].min(), samples[:, 0].max(), 0.1)
# y = np.arange(samples[:, 1].min(), samples[:, 1].max(), 0.1)
# X, Y = np.meshgrid(x, y)
# Z = np.ones_like(X)
# Z = discriminant_fx(samples, )

# surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, antialiased=True)

# ax.scatter(viz_X1[:, 0], viz_X1[:, 1], label="$\omega_1$", c=sample_colors[0])
# ax.scatter(viz_X2[:, 0], viz_X2[:, 1], label="$\omega_2$", c=sample_colors[1])
# ax.scatter(means[0, 0], means[0, 1], marker="*", s=250, label="$\mu_1$", c=mean_colors[0])
# ax.scatter(means[1, 0], means[1, 1], marker="*", s=250, label="$\mu_2$", c=mean_colors[1])
# ax.scatter(x[0, 0], x[0, 1], marker="x", color="red", s=100, label="Test sample 1")
# ax.scatter(x[1, 0], x[1, 1], marker="x", color="magenta", s=100, label="Test sample 2")

ax.set_xlabel("")
ax.set_ylabel("")
ax.set_zlabel("")

# ax.set_aspect(1)
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


[ 0.07640764 -1.3044661 ]
[ 7.07640764 -0.3044661 ]


### Eigenvalue $\lambda$

In [23]:
def compute_eigenvalues(cov):
    b = - cov[0][0] - cov[1][1]
    c = (cov[0][0] * cov[1][1]) - (cov[0][1] * cov[1][0])
    x = sympy.Symbol('x')
    ans = sympy.solveset((x ** 2) + (b * x) + c, x)
    return np.array(list(ans.evalf()), dtype=float)

z = compute_eigenvalues(cov[c])
z

array([2.6, 3.1])

Cross-check with the example in the lecture note

In [24]:
eigenvalues = np.zeros((C, d))
for c in range(C):
    eigenvalues[c] = compute_eigenvalues(cov[c])
eigenvalues

array([[2.6, 3.1],
       [2.6, 3.1]])

### Eigenvector $\phi$

In [25]:
eigenvalues[i]

array([2.6, 3.1])

In [26]:
def compute_eigenvector(cov, eigenvalues):
    eigenvectors = np.zeros((len(eigenvalues), len(eigenvalues)))
    for i, ev in enumerate(eigenvalues):
        denom = (cov[0][0] - eigenvalues[i])
        if denom != 0:
            a = 1
            phi1 = (-cov[0][1] / denom) * a
            phi2 = a
        else:
            phi1 = 1
            phi2 = 0
        eigenvectors[0, i] = phi1
        eigenvectors[1, i] = phi2
    return eigenvectors

In [27]:
compute_eigenvector(cov[c], eigenvalues[c])

array([[-0.,  1.],
       [ 1.,  0.]])

In [28]:
eigenvectors = np.zeros((C, d, d))

for c in range(C):
    for i in range(len(eigenvalues)):
        eigenvectors[c] = compute_eigenvector(cov[c], eigenvalues[c])

In [29]:
eigenvectors[c]

array([[-0.,  1.],
       [ 1.,  0.]])

In [30]:
eigenvalues, eigenvectors

(array([[2.6, 3.1],
        [2.6, 3.1]]),
 array([[[-0.,  1.],
         [ 1.,  0.]],
 
        [[-0.,  1.],
         [ 1.,  0.]]]))

In [31]:
np.linalg.eig(cov[c])

(array([3.1, 2.6]),
 array([[1., 0.],
        [0., 1.]]))

In [32]:
# whitened_eigenvectors = np.array([[-1.80424764150708, 1], [0.554247641507076, 1]])

# fig = plt.figure(figsize=(18, 4))
# cc = 1
# ax = fig.add_subplot(140 + cc + 1)
# ax.plot([0, whitened_eigenvectors[0, 0]], [0, whitened_eigenvectors[0, 1]])
# ax.plot([0, whitened_eigenvectors[1, 0]], [0, whitened_eigenvectors[1, 1]])
# ax.set_aspect(1)
# ax.set_title(f"Eigenvectors of the whitened class {cc+1}")
# plt.show()

In [33]:
def angle(vector_1, vector_2):
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle_rad = np.arccos(dot_product)
    angle = np.rad2deg(angle_rad)
    return angle

### Creating random samples

In [34]:
def gaussian_random(mean, cov, size):
    samples = np.random.normal(loc=0.0, scale=1.0, size=size)
    return (cov @ samples).T + mean

N = 1000

samples = np.zeros((C, N, d))
for c in range(C):
    samples[c] = gaussian_random(mean=[0, 0], cov=np.identity(C), size=(C, N))
# samples = np.random.multivariate_normal(mean=[0, 0], cov=np.identity(2), size=(2, 1000))

In [54]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)
sample_colors = ("tab:blue", "tab:pink")
mean_colors = ("b", "m")
for c in range(C):
    ax.scatter(samples[c, :, 0], samples[c, :, 1], label=f"$\omega_{c+1}$", c=sample_colors[c])
#     ax.scatter(means[c, 0], means[c, 1], marker="*", s=250, label=f"$\mu_{c+1}$", c=mean_colors[c])
ax.set_aspect(1)
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Dewhitening Transformation

<p align="center">
    <img src="https://www.projectrhea.org/rhea/images/thumb/3/35/Fig3_summ_mh.png/685px-Fig3_summ_mh.png!" /> <br />
    <small>https://www.projectrhea.org/rhea/index.php/File:Fig3_summ_mh.png</small>
</p>

In [36]:
def dewhiten(x, mean, eigenvalues, eigenvectors):
    squishing_matrix = np.sqrt(np.identity(len(eigenvalues)) * eigenvalues)
#     y = eigenvectors.T @ squishing_matrix @ x
    y = squishing_matrix @ x
    y = eigenvectors @ y
    y += mean
    return y

# dewhiten(samples[0], means[0], eigenvalues, eigenvectors)

In [37]:
np.linalg.eig(cov[0])

(array([3.1, 2.6]),
 array([[1., 0.],
        [0., 1.]]))

In [38]:
eigenvalues[0], eigenvectors[0]

(array([2.6, 3.1]),
 array([[-0.,  1.],
        [ 1.,  0.]]))

In [39]:
dewhitened_samples = np.zeros_like(samples)

for c in range(C):
    dewhitened_samples[c] = np.array([dewhiten(s, means[c], eigenvalues[c], eigenvectors[c]) for s in samples[c]])

In [40]:
dewhitened_samples.shape

(2, 1000, 2)

In [50]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)
sample_colors = ("tab:blue", "tab:pink")
mean_colors = ("b", "m")
for c in range(C):
    ax.scatter(dewhitened_samples[c, :, 0], dewhitened_samples[c, :, 1], label=f"$\omega_{c+1}$", c=sample_colors[c])
    ax.scatter(means[c, 0], means[c, 1], marker="*", s=250, label=f"$\mu_{c+1}$", c=mean_colors[c])
ax.set_aspect(1)
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
s = np.array([[1, 2], [4, 4]])
eigenvectors[c] * np.sqrt(np.identity(len(eigenvalues[c])) * eigenvalues[c]) @ s

array([[0., 0.],
       [0., 0.]])

In [42]:
dewhitened_samples[0].min(), dewhitened_samples[0].max()

(-3.5589679068359636, 12.695636739787226)

In [58]:
1./cov[c]

<ipython-input-58-8eb2d2a3c3a1>:1: RuntimeWarning: divide by zero encountered in true_divide
  1./cov[c]


array([[0.32258065,        inf],
       [       inf, 0.38461538]])

### Decision Boundary

Discriminant function:
$$ g_i(x) = -\frac{1}{2}(x-\mu_i)^t\Sigma_i^{-1}(x-\mu_i)-\frac{d}{2}\ln (2\pi) - \frac{1}{2} \ln |\Sigma_i| + \ln P(\omega_i) $$

Decision boundary:
$$ g_i(x) = g_j(x) $$
$$ -\frac{1}{2}(x-\mu_i)^t\Sigma_i^{-1}(x-\mu_i)-\frac{d}{2}\ln (2\pi) - \frac{1}{2} \ln |\Sigma_i| + \ln P(\omega_i) = -\frac{1}{2}(x-\mu_j)^t\Sigma_j^{-1}(x-\mu_j)-\frac{d}{2}\ln (2\pi) - \frac{1}{2} \ln |\Sigma_j| + \ln P(\omega_j) $$
$$ -\frac{1}{2}(x-\mu_i)^t\Sigma_i^{-1}(x-\mu_i) - \frac{1}{2} \ln |\Sigma_i| + \ln P(\omega_i) = -\frac{1}{2}(x-\mu_j)^t\Sigma_j^{-1}(x-\mu_j) - \frac{1}{2} \ln |\Sigma_j| + \ln P(\omega_j) $$
From $\Sigma_1 = \Sigma_2 = \begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}$,
$$ -\frac{1}{2}(x-\mu_i)^t\Sigma_i^{-1}(x-\mu_i) + \ln P(\omega_i) = -\frac{1}{2}(x-\mu_j)^t\Sigma_j^{-1}(x-\mu_j) + \ln P(\omega_j) $$
$$ -\frac{1}{2}\left[ x^{\mathsf{T}}\Sigma_i^{-1} x - 2\mu_i^{\mathsf{T}}\Sigma_i^{-1}x + \mu_i^{\mathsf{T}}\Sigma_i^{-1}\mu_i \right] + \ln P(\omega_i) = -\frac{1}{2}\left[ x^{\mathsf{T}}\Sigma_j^{-1} x - 2\mu_j^{\mathsf{T}}\Sigma_j^{-1}x + \mu_j^{\mathsf{T}}\Sigma_j^{-1}\mu_j \right] + \ln P(\omega_j) $$
$$ -\frac{1}{2} x^{\mathsf{T}}\Sigma_i^{-1} x +\frac{1}{2} 2\mu_i^{\mathsf{T}}\Sigma_i^{-1}x -\frac{1}{2}\mu_i^{\mathsf{T}}\Sigma_i^{-1}\mu_i + \ln P(\omega_i) = -\frac{1}{2} x^{\mathsf{T}}\Sigma_j^{-1} x + \frac{1}{2} 2\mu_j^{\mathsf{T}}\Sigma_j^{-1}x -\frac{1}{2} \mu_j^{\mathsf{T}}\Sigma_j^{-1}\mu_j + \ln P(\omega_j) $$
$$ \frac{1}{2} 2\mu_i^{\mathsf{T}}\Sigma_i^{-1}x -\frac{1}{2}\mu_i^{\mathsf{T}}\Sigma_i^{-1}\mu_i + \ln P(\omega_i) = \frac{1}{2} 2\mu_j^{\mathsf{T}}\Sigma_j^{-1}x -\frac{1}{2} \mu_j^{\mathsf{T}}\Sigma_j^{-1}\mu_j + \ln P(\omega_j) $$
$$ \mu_i^{\mathsf{T}}\Sigma_i^{-1}x -\frac{1}{2}\mu_i^{\mathsf{T}}\Sigma_i^{-1}\mu_i + \ln P(\omega_i) = \mu_j^{\mathsf{T}}\Sigma_j^{-1}x -\frac{1}{2} \mu_j^{\mathsf{T}}\Sigma_j^{-1}\mu_j + \ln P(\omega_j) $$
Let $i=1$ and $j=2$, and from $P(\omega_1) = 4P(\omega_2)$, we get
$$ \mu_i^{\mathsf{T}}\Sigma_1^{-1}x -\frac{1}{2}\mu_1^{\mathsf{T}}\Sigma_1^{-1}\mu_1 + \ln 4P(\omega_2) = \mu_2^{\mathsf{T}}\Sigma_2^{-1}x -\frac{1}{2} \mu_2^{\mathsf{T}}\Sigma_2^{-1}\mu_2 + \ln P(\omega_2) $$
$$ \mu_i^{\mathsf{T}}\Sigma_1^{-1}x -\frac{1}{2}\mu_1^{\mathsf{T}}\Sigma_1^{-1}\mu_1 + \ln 4P(\omega_2) - \ln P(\omega_2) = \mu_2^{\mathsf{T}}\Sigma_2^{-1}x -\frac{1}{2} \mu_2^{\mathsf{T}}\Sigma_2^{-1}\mu_2 $$
$$ \mu_i^{\mathsf{T}}\Sigma_1^{-1}x -\frac{1}{2}\mu_1^{\mathsf{T}}\Sigma_1^{-1}\mu_1 + \ln \frac{4P(\omega_2)}{P(\omega_2)} = \mu_2^{\mathsf{T}}\Sigma_2^{-1}x -\frac{1}{2} \mu_2^{\mathsf{T}}\Sigma_2^{-1}\mu_2 $$
$$ \mu_i^{\mathsf{T}}\Sigma_1^{-1}x -\frac{1}{2}\mu_1^{\mathsf{T}}\Sigma_1^{-1}\mu_1 + \ln 4 = \mu_2^{\mathsf{T}}\Sigma_2^{-1}x -\frac{1}{2} \mu_2^{\mathsf{T}}\Sigma_2^{-1}\mu_2 $$
From $\mu_1 = \begin{bmatrix} 7 \\ 1 \end{bmatrix}$, $\mu_2 = \begin{bmatrix} 1 \\ 7 \end{bmatrix}$,
From $\Sigma_1 = \Sigma_2 = \begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}$,
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}^{-1}\vec{x} -\frac{1}{2}\begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}^{-1}\begin{bmatrix} 7 \\ 1 \end{bmatrix} + \ln 4 = \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}^{-1}\vec{x} -\frac{1}{2} \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} 3.1 & 0 \\ 0 & 2.6 \end{bmatrix}^{-1}\begin{bmatrix} 1 \\ 7 \end{bmatrix} $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\vec{x} -\frac{1}{2}\begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} 7 \\ 1 \end{bmatrix} + \ln 4 = \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\vec{x} -\frac{1}{2} \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} 1 \\ 7 \end{bmatrix} $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} -\frac{1}{2}\begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} 7 \\ 1 \end{bmatrix} + \ln 4 = \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} -\frac{1}{2} \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} 1 \\ 7 \end{bmatrix} $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} -\frac{1}{2}\begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{7}{3.1} \\ \frac{1}{2.6} \end{bmatrix} + \ln 4 
= \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} -\frac{1}{2} \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} \\  \frac{7}{2.6} \end{bmatrix} $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} -\frac{1}{2} (\frac{7 \cdot 7}{3.1} + \frac{1}{2.6}) + \ln 4 
= \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} -\frac{1}{2} (\frac{1}{3.1} + \frac{7 \cdot 7}{2.6}) $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} - 8.0955 + \ln 4 
= \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} - 9.5844 $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} + 2.8752
= \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{1}{3.1} & 0 \\ 0 & \frac{1}{2.6} \end{bmatrix}\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} $$
$$ \begin{bmatrix} 7 & 1 \end{bmatrix}\begin{bmatrix} \frac{x_1}{3.1} \\ \frac{x_2}{2.6} \end{bmatrix} + 2.8752
= \begin{bmatrix} 1 & 7 \end{bmatrix}\begin{bmatrix} \frac{x_1}{3.1} \\ \frac{x_2}{2.6} \end{bmatrix} $$
$$ \frac{7 \cdot x_1}{3.1} + \frac{x_2}{2.6} + 2.8752
= \frac{x_1}{3.1} + \frac{7 \cdot x_2}{2.6} $$

Multiply both sides by $3.1 \times 2.6 = 8.06$,
$$ (2.6 \cdot 7 \cdot x_1) + (3.1 \cdot x_2) + (8.06 \cdot 2.8752) = (2.6 \cdot x_1) + (3.1 \cdot 7 \cdot x_2) $$
$$ 18.2 x_1 + 3.1x_2 + 23.1741 = 2.6 x_1 + 21.7 x_2 $$
$$ 15.6 x_1 + 23.1741 = 18.6 x_2 $$
$$ 0.8387x_1 + 1.2459 = x_2 $$

In [59]:
def decision_boundary(x1):
    return (0.8387 * x1) + 1.2459

In [63]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
surface_colors = ("Blues", "pink")
mean_colors = ("b", "m")

OFFSET = -10

for c in range(C):
#     x = np.arange(-10, 12, .1)
#     y = np.arange(-4, 16, .1)
    x = np.arange(dewhitened_samples[c, :, 0].min(), dewhitened_samples[c, :, 0].max(), .1)
    y = np.arange(dewhitened_samples[c, :, 1].min(), dewhitened_samples[c, :, 1].max(), .1)
    X, Y = np.meshgrid(x, y)
    zs = np.array([discriminant_fx(s, means[c], cov[c], priors[c]) for s in np.array(list(zip(np.ravel(X), np.ravel(Y))))])
    Z = zs.reshape(X.shape)
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, antialiased=True, cmap=surface_colors[c])
    
    ax.scatter(means[c, 0], means[c, 1], OFFSET, marker="*", s=250, label=f"$\mu_{c+1}$", c=mean_colors[c])
    
    theta = np.linspace(0, 2*np.pi, 100)

    major_idx = np.argmax(eigenvalues[c])
#         minor_idx = np.argmin(eigenvalues[c])
    # HOTFIX: argmax = argmin causes a problem
    minor_idx = 1 if major_idx == 0 else 0
    alpha = np.arctan2(eigenvectors[c, 1, major_idx], eigenvectors[c, 0, major_idx])
    major_r_x = eigenvalues[c, major_idx]
    major_r_y = eigenvalues[c, minor_idx]
    AXIS_COLORS = ["black", "grey"]
    axis_colors = [AXIS_COLORS[major_idx], AXIS_COLORS[minor_idx]]
    
    e_X = major_r_x * np.cos(theta) * np.cos(alpha) - major_r_y * np.sin(theta) * np.sin(alpha) + means[c, 0]
    e_Y = major_r_x * np.cos(theta) * np.sin(alpha) + major_r_y * np.sin(theta) * np.cos(alpha) + means[c, 1]
    e_Z = np.zeros_like(e_X) + OFFSET
    ax.plot(e_X, e_Y, e_Z, color="black", linestyle="--")

x_min, x_max = ax.get_xlim()
x = np.linspace(x_min, x_max, 100)
y = decision_boundary(x)
ax.plot(x, y, OFFSET, label="Decision boundary", color="black")

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
ax.set_zlabel("")

ax.grid(False)

ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
zs = np.array([discriminant_fx(s, means[0], cov[0], priors[0]) for s in np.array(list(zip(np.ravel(X), np.ravel(Y))))])
Z = zs.reshape(X.shape)
# g1 = np.array([discriminant_fx(s, means[0], cov, priors[0]) for s in dewhitened_samples_c1])

In [ ]:
np.ravel(X)[0:3], np.ravel(Y)[0:3]

In [ ]:
np.array(list(zip(np.ravel(X), np.ravel(Y)))).shape